In [1]:
dir_read = '/Users/rachellehorwitz/Documents/VTAlert/data/filtered02/'
#dir_read = '/Users/rachellehorwitz/Documents/VTAlert/over18_eicu/'

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import streamlit as st

In [2]:
# Read files from csv
patient = pd.read_csv(dir_read + 'patient.csv')
admissiondx = pd.read_csv(dir_read + 'admissiondx.csv')
diagnosis = pd.read_csv(dir_read + 'diagnosis.csv')
ptusid_pos = pd.read_csv(dir_read + 'pos_ptunitstayid.csv')

# Create dataframe of patientunitstayids in negative class
tmp = patient[~patient['patientunitstayid'].isin(ptusid_pos['patientunitstayid'])]
ptusid_neg = tmp[['patientunitstayid']]

# Create labels
ptusid_neg.loc[:,'label'] = 0
ptusid_pos.loc[:,'label'] = 1

# Add labels to "patient" dataframe and move 'label' column to left
ptusid_w_label = pd.concat([ptusid_pos, ptusid_neg])
ptusid_w_label.shape[0]
ptusid_w_label = ptusid_w_label.set_index('patientunitstayid')
patient = patient.set_index('patientunitstayid')
patient = patient.join(ptusid_w_label)

# Move 'label' column to left
cols = list(patient.columns)
cols = [cols[-1]] + cols[:-1]
patient = patient[cols]

/Users/rachellehorwitz/opt/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/rachellehorwitz/opt/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [3]:
patient.head(5)

,label,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141168,0,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
141194,0,128941,Male,68,Caucasian,73,92,"Sepsis, renal/UTI (including bladder)",180.3,18:18:40,...,Floor,1,admit,73.9,76.7,15:31:00,4813,Floor,Alive,002-5276
141197,0,128943,Male,71,Caucasian,67,109,"Sepsis, pulmonary",162.6,20:21:00,...,Emergency Department,1,admit,102.1,102.1,22:00:00,74,Step-Down Unit (SDU),Alive,002-37665
141203,0,128948,Female,77,Caucasian,66,90,"Arrest, respiratory (without cardiac arrest)",160.0,22:23:00,...,Floor,1,admit,70.2,70.3,03:48:00,1869,Floor,Alive,002-23234
141208,0,128952,Female,25,Caucasian,73,93,"Overdose, sedatives, hypnotics, antipsychotics...",172.7,11:23:00,...,Emergency Department,1,admit,95.3,95.3,23:24:00,720,Home,Alive,002-34521


In [4]:
# Define X and y
X = patient[['age', 'admissionweight']]
y = patient['label']

# Partition and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [5]:
X.head()

,age,admissionweight
patientunitstayid,,
141168,70,84.3
141194,68,73.9
141197,71,102.1
141203,77,70.2
141208,25,95.3


In [6]:
X_test

,age,admissionweight
patientunitstayid,,
1224402,84,94.1
1116074,50,91.0
1593370,71,58.5
1842744,75,125.1
3182842,39,48.2
...,...,...
1341158,80,68.7
2902515,46,150.2
2729632,77,64.4


In [7]:
# X_test.iloc[0]

In [8]:
# age = 33
# admissionweight = 50
# input_data = {'age': [age], 'admissionweight': [admissionweight]}
# X_test = pd.DataFrame(input_data, columns=['age', 'admissionweight'])


In [9]:
# X_test.iloc[0]

In [10]:

logisticRegr_sc = make_pipeline(StandardScaler(), LogisticRegression(class_weight='balanced'))
logisticRegr_sc.fit(X_train, y_train)

# Test model and calculate outputs
predictions_sc = logisticRegr_sc.predict(X_test)
prob_sc = logisticRegr_sc.predict_proba(X_test)[:,1]
lprob_sc = logisticRegr_sc.predict_log_proba(X_test)[:,1]
scores_sc = logisticRegr_sc.score(X_test, y_test)
print(scores_sc)

0.5396728701532089


In [11]:
np.unique(predictions_sc)

array([0, 1])

In [12]:
np.where(predictions_sc==1)

(array([    0,     3,     5, ..., 32822, 32823, 32827]),)

In [13]:
print('prob_sc:  min = ' + str(np.min(prob_sc)))
print('\t    max = ' + str(np.max(prob_sc)))

prob_sc:  min = 0.3729853784561912
	    max = 0.9729283126050523


In [14]:
prob_sc[0]

0.5296695506882767

In [15]:
# Baseline if always predict 0: wrong only for when y is 1
N0_bl = patient[patient['label']==0].shape[0]
N1_bl = patient[patient['label']==1].shape[0]

print(str(N0_bl))
print(str(N1_bl))
print(N0_bl/(N0_bl + N1_bl))

162165
1987
0.987895365271212
